# 1. Introdução ao Altair

[Altair](https://altair-viz.github.io/) é um módulo no Python de visualização estatística declarativa. Essa biblioteca oferece uma gramática de visualização concisa e poderosa para uma rápida construção dos códigos. Possui também  uma grande variedade de gráficos estatísticos.

O termo *“declarativa”* quer dizer que você pode ter uma especificação de alto nível sobre *quais* características serão incluídas na visualização em termos de *dados, marcas gráficas ( ou marcações gráficas),* e *canais de codificação*, ao invés de ter de especificar *como* implementar a visualização em termos de for-loops, comandos de desenho em linguagem de baixo nível e outros. A ideia central é você poder fazer conexões entre os campos de dados e canais de codificação visuais, como os eixos cartesianos ou as cores da sua visualização. O resto dos detalhes da plotagem são computados automaticamente. Se baseando nessa ideia de plotagem declarativa, uma surpreendente gama de visualizações, das simples às mais sofisticadas, podem ser criadas usando uma gramática concisa.

A biblioteca Altair é baseada no [Vega-Lite](https://vega.github.io/vega-lite/), uma linguagem de alto nível de gráficos interativos. Altair oferece uma interface [API (Interface de Programação de Aplicação)](https://en.wikipedia.org/wiki/API) em Python que gera especificações no Vega-Lite no formato [JSON (Notação de Objeto em Javascript)](https://en.wikipedia.org/wiki/JSON). Ambientes como o Jupyter Notebooks, Jupyter Lab e o Google Colab pegam e renderizam essas especificações diretamente no navegador web. Para saber mais sobre a motivação e os conceitos básicos sobre o Altair e Vega-Lite, assista o vídeo sobre a [apresentação do Vega-Lite no OpenVisConf 2017.](https://www.youtube.com/watch?v=9uaHRWj04D4)

Este capítulo vai te auxiliar no processo básico de criar visualizações no Altair. Primeiramente, você precisará ter certeza que o pacote do Altair e suas dependências estão instaladas (para mais informações, acesse a [documentação de instalação do Altair](https://altair-viz.github.io/getting_started/installation.html)), ou que você já esteja utilizando um ambiente de notebook que inclua os pacotes pré-instalados.

*Este capítulo faz parte do [currículo de visualização de dados.](https://github.com/uwdata/visualization-curriculum)*


## <font color="gray">1.1</font> Importações
---
Para começarmos, precisamos importar as bibliotecas necessárias: Pandas para data frames e o Altair para a visualização.

In [ ]:
import pandas as pd
import altair as alt

## <font color="gray">1.2</font> Renderizações
---
Dependendo do seu ambiente, você talvez precisará especificar um [renderizador](https://altair-viz.github.io/user_guide/display_frontends.html) para o Altair. Se você estiver utilizando o **Jupyter Lab, Jupyter Notebook** ou o **Google Colab** conectado a internet, você não precisa fazer nada nessa parte. Se não for o caso, por favor leia a documentação para [a exibição de gráficos no Altair.](https://altair-viz.github.io/user_guide/display_frontends.html)

## <font color="gray">1.3</font> Dados
---
Dados no Altair são construídos em torno do data frame do Pandas, que consiste em um conjunto de colunas de dados nomeadas. Também iremos nos referir a colunas de dados como campos de dados.

Usando o Altair, conjuntos de dados são comumente providenciados como data frames. De forma alternativa, a biblioteca também aceita um URL para carregar um conjunto de dados acessível por network. Como veremos, as colunas de um data frame são parte essencial da plotagem no Altair.

Geralmente usaremos conjuntos de dados do repositório vega-datasets. Alguns desses conjuntos estão disponíveis diretamente no Pandas como data frames:


In [ ]:
from vega_datasets import data  # importa vega_datasets
cars = data.cars()              # carrega os dados de carros como um data frame do Pandas
cars.head()                     # mostra as primeiras cinco linhas

Name  Miles_per_Gallon  Cylinders  Displacement  \
0  chevrolet chevelle malibu              18.0          8         307.0   
1          buick skylark 320              15.0          8         350.0   
2         plymouth satellite              18.0          8         318.0   
3              amc rebel sst              16.0          8         304.0   
4                ford torino              17.0          8         302.0   

   Horsepower  Weight_in_lbs  Acceleration       Year Origin  
0       130.0           3504          12.0 1970-01-01    USA  
1       165.0           3693          11.5 1970-01-01    USA  
2       150.0           3436          11.0 1970-01-01    USA  
3       150.0           3433          12.0 1970-01-01    USA  
4       140.0           3449          10.5 1970-01-01    USA

*No data frame acima, há variáveis de Nome, Milhas por galão, Cilíndros, Cilindradas, Potência de Cavalos, Peso em libras, Aceleração, Ano e Origem.*

Conjuntos de dados no vega-datasets também podem ser acessados via URLs:

In [ ]:
data.cars.url

'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/cars.json'

URLs de conjuntos de dados podem ser passados diretamente ao Altair (em formatos como JSON ou [CSV](https://en.wikipedia.org/wiki/Comma-separated_values)), ou carregados em um data frame do Pandas:

In [ ]:
pd.read_json(data.cars.url).head() # carrega os dados de JSON em um data frame

Name  Miles_per_Gallon  Cylinders  Displacement  \
0  chevrolet chevelle malibu              18.0          8         307.0   
1          buick skylark 320              15.0          8         350.0   
2         plymouth satellite              18.0          8         318.0   
3              amc rebel sst              16.0          8         304.0   
4                ford torino              17.0          8         302.0   

   Horsepower  Weight_in_lbs  Acceleration        Year Origin  
0       130.0           3504          12.0  1970-01-01    USA  
1       165.0           3693          11.5  1970-01-01    USA  
2       150.0           3436          11.0  1970-01-01    USA  
3       150.0           3433          12.0  1970-01-01    USA  
4       140.0           3449          10.5  1970-01-01    USA

Para mais informações sobre data frames - e algumas transformações úteis para preparar data frames do Pandas para plotagem com o Altair! - veja a documentação de [especificação de dados com Altair.](https://altair-viz.github.io/user_guide/data.html)

### <font color="gray">1.3.1 </font> Dados Meteorológicos
---
Visualização estatística com o Altair começa com ["pequenos"](https://vita.had.co.nz/papers/tidy-data.html) data frames. Aqui, começaremos criando um data frame simples (chamaremos de `df`) contendo a precipitação média (`precip`) de uma dada cidade(`city`) em um determinado mês(`month`):

In [ ]:
df = pd.DataFrame({
    'city': ['Seattle', 'Seattle', 'Seattle', 'New York', 'New York', 'New York', 'Chicago', 'Chicago', 'Chicago'], # coluna de cidades
    'month': ['Apr', 'Aug', 'Dec', 'Apr', 'Aug', 'Dec', 'Apr', 'Aug', 'Dec'], # coluna de meses
    'precip': [2.68, 0.87, 5.31, 3.94, 4.13, 3.58, 3.62, 3.98, 2.56] # coluna de precipitação
})

df

city month  precip
0   Seattle   Apr    2.68
1   Seattle   Aug    0.87
2   Seattle   Dec    5.31
3  New York   Apr    3.94
4  New York   Aug    4.13
5  New York   Dec    3.58
6   Chicago   Apr    3.62
7   Chicago   Aug    3.98
8   Chicago   Dec    2.56

## <font color="gray">1.4 </font> O objeto gráfico

---
O objeto fundamental utilizado no Altair é o gráfico (em inglês, `chart`), que recebe um data frame como um argumento:

In [ ]:
chart = alt.Chart(df)

Até agora, nós definimos o objeto ‘gráfico’ (`chart`) e passamos o data frame que geramos acima. Ainda não demos nenhum comando para o gráfico seguir com essas informações.

## <font color="gray">1.5 </font> Marcas e Codificações

---
Com  o gráfico em mãos, podemos agora especificar como queremos visualizar os dados. Primeiro indicaremos que tipo de marca gráfica (forma geométrica) gostaríamos de representar os dados. Selecionaremos o atributo marca (`mark`) no objeto do gráfico usando o método `Chart.mark_*`.

Por exemplo, podemos mostrar os dados usando o comando `Chart.mark_point`:


In [ ]:
alt.Chart(df).mark_point()

alt.Chart(...)

Aqui a renderização consiste em um ponto por linha no conjunto de dados, todos plotados um em cima do outro pois não especificamos ainda a posição desses pontos.

Para separarmos visualmente esses pontos, podemos mapear vários *canais de codificação*, ou apenas *canais*, para os campos de conjunto de dados. Por exemplo, poderíamos codificar o campo `city `(cidade) os dados utilizando o canal `y`, que representa a posição dos pontos no eixo `y`. Para especificarmos isso, usamos o método `encode`(codificar):

In [ ]:
alt.Chart(df).mark_point().encode( # marca de codificação de gráfico de barras
  y='city', # eixo y - cidade
)

alt.Chart(...)

O método `encode()` constrói um mapeamento por valores chave entre os canais de codificação ( canais `x`, `y`,`cor`, `shape`(formato),` size`(tamanho), *etc*.) para campos no conjunto de dados, acessados pelo nome do campo. Para os data frames do Pandas, o Altair automaticamente determina o tipo de dado apropriado na coluna mapeada, que nesse caso é do tipo *nominal*, indicando valores categóricos sem ordem.

Entretanto, mesmo que já tenhamos separado os dados por um atributo, nós ainda temos múltiplos pontos se sobrepondo em cada tipo de categoria. Vamos continuar separando esses pontos adicionando um canal de codificação `x`, mapeado no campo `’precip’`:

In [ ]:
alt.Chart(df).mark_point().encode( # marca de codificação de gráfico de barras
    x='precip', # eixo x - precipitação
    y='city' # eixo y - cidade
)

alt.Chart(...)

*Seattle exibe tanto os meses que menos choveram como o que mais choveram!*

O tipo de dado no campo `’precip’` é inferido novamente pelo Altair, e dessa vez é tratado como um tipo *quantitativo* (isso é, um número real). Vemos que as linhas do grid e o título dos eixos também foram automaticamente adicionados.

Acima nós especificamos os pares chave-valor usando argumentos chave ( `x=’precip’`). Além disso, Altair nos dá métodos de construção para definições de codificação, usando a sintaxe `alt.X(’precip’)`. Essa alternativa é útil para colocar mais parâmetros em uma codificação, como veremos mais adiante no capítulo.

In [ ]:
alt.Chart(df).mark_point().encode( # marca de codificação de gráfico de barras
    alt.X('precip'), # eixo x - precipitação
    alt.Y('city') # eixo y - cidade
)

alt.Chart(...)

Os dois estilos de especificar codificações podem ser intercalados: `x=’precip’, alt.Y(’city’)` são entradas válidas na função `encode`.

Nos exemplos, o tipo de dado de cada campo é inferido automaticamente baseado no tipo dentro do data frame do Pandas. Indicar explicitamente o tipo de dado ao Altair também é possível adicionando o nome do campo.



* `’b:N’`- Indica um tipo *nominal* ( dados categorizados sem ordem )
* `’b:O’`- Indica um tipo *ordinal* ( dados ordenados )
* `’b:Q’`- Indica um tipo *quantitativo* ( dados numéricos com magnitudes significativas )
* `’b:T’`- Indica um tipo *temporal* ( dados de tempo/data )

Por exemplo, `alt.X(‘precip:N’)`.

Notação explícita dos tipos de dados é necessária quando os dados são carregados de um URL externo diretamente ao Vega-Lite ( pulando o procedimento do Pandas por completo ), ou quando nós queremos usar um tipo que difere do tipo automaticamente inferido.

O que você acha que vai acontecer com o nosso gráfico acima quando tratarmos `precip` como um valor nominal ou ordinal, em vez de um valor quantitativo? *Modifique o código acima e descubra!*

Daremos uma olhada melhor nos tipos de dados e canais de codificação no próximo capítulo do [currículo de visualização de dados](https://github.com/uwdata/visualization-curriculum#data-visualization-curriculum).





## <font color="gray">1.6</font> Transformação de Dados: Agrupamento

---
Para poder ter mais flexibilidade em como os dados são visualizados, Altair tem embutido uma sintaxe para o *agrupamento* de dados. Por exemplo, podemos computar a média de todos os valores especificando a função de agrupamento junto com o nome do campo.

In [ ]:
alt.Chart(df).mark_point().encode( # marca de codificação de gráfico de barras
    x='average(precip)', # eixo x - média de precipitação
    y='city' # eixo y - cidade
)

alt.Chart(...)

Agora dentro de cada categoria no eixo x, vemos um ponto refletindo a média dos valores daquela categoria.

*Será que Seattle realmente tem a menor precipitação nessas cidades? (Realmente tem!) Mesmo assim, como essa visualização passa a informação errada? Quais meses foram incluídos? O que conta como precipitação?*

Altair suporta uma variedade de funções de agrupamento, incluindo `count`(contador), `min`(mínimo), `max`(máximo), `average`(média), `median`(mediana), e  `stdev`(desvio padrão). Em um próximo capítulo, teremos um tour sobre transformações de dados, incluindo agrupamento, ordenação, filtração e criação de novos campos derivados usando fórmulas.

## <font color="gray">1.7</font> Mudando o Tipo de Marca

---

Considere que queremos representar nossos valores agrupados usando barras retangulares em vez de pontos circulares. Podemos fazer isso trocando o método `Chart.mark_point` por `Chart.mark_bar`






In [ ]:
alt.Chart(df).mark_bar().encode( # marca de codificação de gráfico de barras
    x='average(precip)', # eixo x - média da precipitação
    y='city' # eixo y - cidades
)

alt.Chart(...)

O campo nominal `a` foi mapeado no eixo `y`, resultando no gráfico de barra acima. Para deixarmos a barra na vertical, basta trocar o eixo `y` com o eixo `x`.

In [ ]:
alt.Chart(df).mark_bar().encode( # marca de codificação de gráfico de barras, aqui os eixos são o contrário do de cima
    x='city', #eixo x - cidades
    y='average(precip)' #eixo y - média de precipitação
)

alt.Chart(...)

## <font color="grey">1.8</font> Customizando uma Visualização


---

Por padrão, o Pandas e o Vega-Lite fazem algumas escolhas sobre  de propriedades da visualização, mas isso pode ser mudado usando métodos de customização da aparência da visualização. Por exemplo, podemos especificar os títulos dos eixos usando o atributo `axis` (eixo) do canal de classes, podemos modificar as propriedades de escalas usando o atributo  `scale`(escala), e também podemos especificar a cor das marcas pelo atributo `color`(cor) dos métodos do `Chart.Mark_*` de qualquer [CSS color string](https://developer.mozilla.org/en-US/docs/Web/CSS/color_value):



In [ ]:
alt.Chart(df).mark_point(color='firebrick').encode( # marca de codificação de gráfico de pontos
  alt.X('precip', scale=alt.Scale(type='log'), axis=alt.Axis(title='Log-Scaled Values')), # eixo x são as precipitações em escala logarítmica
  alt.Y('city', axis=alt.Axis(title='Category')), # eixo y são as cidades
)

alt.Chart(...)

Um módulo subsequente vai explorar as várias opções disponíveis para as escalas, os eixos e as legendas para criar gráficos customizados.

## <font color="grey">1.9</font> Múltiplas Visualizações


---


Como vimos acima, o objeto `Chart`(gráfico) no Altair representa uma plotagem com um único tipo de marca. Mas e para diagramas mais complicados, envolvendo múltiplas camadas ou gráficos? Usando um conjunto de *composição de visualização*, o Altair pode receber múltiplas definições de gráficos e combinar todas para criar visualizações mais complexas.

Inicialmente, vamos plotar o conjunto de dados de carros em um gráfico de linha mostrando a média de milhas por litro no ano de fabricação:




In [ ]:
alt.Chart(cars).mark_line().encode( # marca de codificação de gráfico de linha
    alt.X('Year'), # eixo x - ano
    alt.Y('average(Miles_per_Gallon)') # eixo y - média das milhas por galão
)

alt.Chart(...)

Para complementarmos esse gráfico, poderíamos adicionar marcadores `circle`(círculo) em cada ponto. (O marcador do `circle` é apenas um atalho para marcas de pontos que usam círculos preenchidos.)

Podemos começar definindo cada gráfico separadamente: primeiro a plotagem da linha, depois a plotagem dos pontos. Podemos usar o operador `layer`(camada) para combinar os dois em uma camada do gráfico. Aqui nós usamos o operador `+`(mais) como atalho para usar o modo de camadas:

In [ ]:
line = alt.Chart(cars).mark_line().encode( # linhas da marca de codificação de gráfico de linha
    alt.X('Year'), # eixo x - ano
    alt.Y('average(Miles_per_Gallon)') # eixo y - média das milhas por galão
)

point = alt.Chart(cars).mark_circle().encode( # pontos da marca de codificação de gráfico de linha
    alt.X('Year'),
    alt.Y('average(Miles_per_Gallon)')
)

line + point # soma dos dois gera um gráfico com ambas características

alt.LayerChart(...)


Podemos também criar esse gráfico *reutilizando* e *modificando* uma definição de gráfico anterior! Em vez de simplesmente reescrever o gráfico, começaremos com um gráfico de linha, e aí chamaremos o método `mark_point` para gerar uma nova definição de gráfico com uma marca diferente:

In [ ]:
mpg = alt.Chart(cars).mark_line().encode( # linhas da marca de codificação de gráfico de linha
    alt.X('Year'),
    alt.Y('average(Miles_per_Gallon)')
)

mpg + mpg.mark_circle() #gráfico com marcações adicionadas

alt.LayerChart(...)

*(A necessidade de colocar pontos em linhas é tão comum, que a marca `line`(linha) também inclui um atalho para gerar uma nova camada para você. Tente adicionar o argumento `point=True` para o método `mark_line`!)*


Agora, e se a gente quisesse ver esse gráfico junto com outros plots, como a média de potência dos carros ao longo dos anos?

Podemos usar os operadores de concatenação para criar vários gráficos um do lado do outro, ficando na vertical ou na horizontal. Aqui, vamos usar o operador `|` ( pipe ) para fazer uma concatenação horizontal de dois gráficos:

In [ ]:
hp = alt.Chart(cars).mark_line().encode( # linhas da marca de codificação de gráfico de linha
    alt.X('Year'), # eixo x - ano
    alt.Y('average(Horsepower)') # eixo y - média da potência
)

(mpg + mpg.mark_circle()) | (hp + hp.mark_circle()) #concatenação dos dois gráficos

alt.HConcatChart(...)

*Podemos ver que, nesse conjunto de dados, entre os anos 70 e o início dos anos 80 a média de eficiência do combustível melhorou enquanto a média da potência diminuiu.*


Um próximo capítulo focará na *composição de visualizações*, incluindo não apenas gerenciamento de camadas e concatenação, mas também o operador `facet` para dividir os dados em sub-plots e o operador `repeat` (repetir) para gerar de forma concisa gráficos concatenados de um template.

##  <font color="grey">1.10</font> Interatividade


---

Além da plotagem básica e composição de visualizações, uma das coisas mais legais do Altair e do Vega-Lite é o suporte para as interações.

Para criar um simples plot interativo que possui funções de movimentação e ampliação, podemos chamar o método `interactive()` do objeto `Chart`. No gráfico abaixo, clique e arraste para *movimentar-se* e use o scroll do mouse para dar *zoom*:

In [ ]:
alt.Chart(cars).mark_point().encode( # marca de codificação de gráfico de pontos
    x='Horsepower', # eixo x - potência
    y='Miles_per_Gallon', # eixo y - milhas por galão
    color='Origin', # cores dos pontos dependem da origem: pontos azuis são da Europa, pontos amarelos do Japão e pontos vermelhos dos Estados Unidos
).interactive()

alt.Chart(...)

Para dar mais detalhes sobre o uso do mouse, podemos usar o canal de codificador `tooltip`:

In [ ]:
alt.Chart(cars).mark_point().encode(
    x='Horsepower', # eixo x - potência
    y='Miles_per_Gallon', # eixo y - milhas por galão
    color='Origin', # cor - origem
    tooltip=['Name', 'Origin'] # mostram o nome e a origem em um pop-up
).interactive()

alt.Chart(...)

Para mais interações complexas, como gráficos conectados e cross-filtering, Altair oferece uma abstração *seletiva* para definir seleções interativas e adicioná-las como componente do gráfico. Nós vamos ver isso com mais detalhes em um próximo capítulo.

Abaixo segue um exemplo mais complexo. O histograma superior mostra o contador de carros por ano e usa uma seleção interativa para modificar a opacidade dos pontos no scatter plot em baixo, que por sua vez mostra a potência do carro dada a quantidade de milhas.

*Arraste um intervalo no gráfico de cima e veja como ele afeta os pontos no gráfico de baixo. Enquanto você examina o código, **não se preocupe se algumas partes não fizerem sentido ainda!** Isso é um exemplo aspiracional, e nós vamos preencher os detalhes durante a progressão dos próximos capítulos.*

In [ ]:
# cria um intervalo de seleção sobre o eixo x
brush = alt.selection_interval(encodings=['x'])

# determina a opacidade baseado no intervalo "brush"
opacity = alt.condition(brush, alt.value(0.9), alt.value(0.1))

# uma visão geral do histograma de carros por ano
# adiciona o intervalo "brush" para selecionar os carros ao longo do tempo
overview = alt.Chart(cars).mark_bar().encode(
    alt.X('Year:O', timeUnit='year', # extrai a unidade de ano, trata como ordinal
      axis=alt.Axis(title=None, labelAngle=0) # sem título, sem rótulo de ângulo
    ),
    alt.Y('count()', title=None), # contador, sem título nos eixos
    opacity=opacity
).add_selection(
    brush      # adiciona intervalo de seleção do "brush" no gráfico
).properties(
    width=400, # define a largura do gráfico em 400 pixels
    height=50  # define a altura do gráfico em 50 pixels
)

# um scatterplot detalhado de potência x milhas
# modula a opacidade do ponto baseado na seleção do "brush"
detail = alt.Chart(cars).mark_point().encode(
    alt.X('Horsepower'),
    alt.Y('Miles_per_Gallon'),
    # define a opacidade baseado na seleção do "brush"
    opacity=opacity
).properties(width=400)  # ajusta a largura do gráfico para ficar do mesmo tamanho que o primeiro

# concatena verticalmente (vconcat) os gŕaficos usando o operador '&'
overview & detail

<ipython-input-21-822ca9276fa5>:15: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  ).add_selection(


alt.VConcatChart(...)

##  <font color="grey">1.11</font> Além dos gráficos: Examinando o output JSON


---

Como uma API do Python para o Vega-Lite, o propósito principal do Altair é de converter especificações de plotagem em uma string em JSON que funciona nos esquemas do Vega-Lite. Usando o método `Chart.to_json`, podemos inspecionar as especificações do JSON que o Altair está exportando e enviando ao Vega-Lite:

In [ ]:
chart = alt.Chart(df).mark_bar().encode(
    x='average(precip)',
    y='city',
)
print(chart.to_json()) # imprime as especificações do JSON

{
  "$schema": "https://vega.github.io/schema/vega-lite/v5.20.1.json",
  "config": {
    "view": {
      "continuousHeight": 300,
      "continuousWidth": 300
    }
  },
  "data": {
    "name": "data-8e72c2f67818e64f2c6d729f1a903405"
  },
  "datasets": {
    "data-8e72c2f67818e64f2c6d729f1a903405": [
      {
        "city": "Seattle",
        "month": "Apr",
        "precip": 2.68
      },
      {
        "city": "Seattle",
        "month": "Aug",
        "precip": 0.87
      },
      {
        "city": "Seattle",
        "month": "Dec",
        "precip": 5.31
      },
      {
        "city": "New York",
        "month": "Apr",
        "precip": 3.94
      },
      {
        "city": "New York",
        "month": "Aug",
        "precip": 4.13
      },
      {
        "city": "New York",
        "month": "Dec",
        "precip": 3.58
      },
      {
        "city": "Chicago",
        "month": "Apr",
        "precip": 3.62
      },
      {
        "city": "Chicago",
        "month": "Aug",

Note que aqui o `encode(x=’average(precip)’)` foi expandido a uma estrutura em JSON com o nome do campo ( `field` ), o tipo do dado  ( `type` ), e um campo do método `aggregate`(agregar). O `encode(y='city') ` foi expandido de forma análoga.

Como vimos antes, os atalhos de sintaxe do Altair incluem um jeito de especificar o tipo de campo:


In [ ]:
x = alt.X('average(precip):Q')
print(x.to_json()) # aqui, ele está pegando a variavel 'average(precip)' e vendo suas especificações em JSON

{
  "aggregate": "average",
  "field": "precip",
  "type": "quantitative"
}


Esse atalho é equivalente a escrever os atributos pelo nome:

In [ ]:
x = alt.X(aggregate='average', field='precip', type='quantitative')
print(x.to_json())

{
  "aggregate": "average",
  "field": "precip",
  "type": "quantitative"
}


## <font color="grey">1.12</font> - Publicando uma visualização


---

Após visualizar seus dados, talvez você queira publicar eles em algum lugar na internet. Isso pode ser feito de forma rápida usando o [pacote embutido de Javascript dentro do vega](https://github.com/vega/vega-embed).
Um exemplo simples apenas usando um documento HTML pode ser gerado para qualquer gráfico usando o método `Chart.save`:


```python
chart = alt.Chart(df).mark_bar().encode(
    x='average(precip)',
    y='city',
)
chart.save('chart.html')
```



O template de HTML básico produz um output que parece desse jeito, onde as especificações do JSON da sua plotagem produzidas pelo `Chart.to_json` devem ser guardadas na variável `spec` do JavaScript:



```html
<!DOCTYPE html>
<html>
  <head>
    <script src="https://cdn.jsdelivr.net/npm/vega@5"></script>
    <script src="https://cdn.jsdelivr.net/npm/vega-lite@4"></script>
    <script src="https://cdn.jsdelivr.net/npm/vega-embed@6"></script>
  </head>
  <body>
  <div id="vis"></div>
  <script>
    (function(vegaEmbed) {
      var spec = {}; /* As especificações do seu gráfico geram um output .JSON */
      var embedOpt = {"mode": "vega-lite"}; /* Opções para o embedding */

      function showError(el, error){
          el.innerHTML = ('<div style="color:red;">'
                          + '<p>JavaScript Error: ' + error.message + '</p>'
                          + "<p>This usually means there's a typo in your chart specification. "
                          + "See the javascript console for the full traceback.</p>"
                          + '</div>');
          throw error;
      }
      const el = document.getElementById('vis');
      vegaEmbed("#vis", spec, embedOpt)
        .catch(error => showError(el, error));
    })(vegaEmbed);
  </script>
</body>
</html>
```



O método `Chart.save` nos dá um jeito conveniente de salvar o HTML em um arquivo. Para mais informações sobre incorporar o Altair e/ou o Vega-Lite, veja a [documentação para projetos incorporados no Vega](https://github.com/vega/vega-embed).

##  <font color="grey">1.13</font> Próximos passos


---

🎉 Parabéns, você completou o capítulo de introdução ao Altair! No próximo capítulo, nós vamos explorar a criação de visualizações usando modelos de tipos de dados do Altair, marcas gráficas, e canais gráficos de codificação.